# Pricing Opzioni con Barriera con formule chiuse alla BSM

In [25]:
import os, sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

# Imposto la root del progetto a partire dalla posizione di questo file
PROJECT_ROOT = Path.cwd().parent
SRC_DIR = PROJECT_ROOT / "src"
DATA_RESULTS = PROJECT_ROOT / "data" / "results"
DATA_RESULTS.mkdir(parents=True, exist_ok=True)

# Aggiungo la cartella src al path per poter importare i moduli personalizzati
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

In [20]:
import importlib, bsm, barrier_bs
importlib.reload(barrier_bs)
from barrier_bs import call_up_and_out, call_up_and_in
from bsm import bs_price

S, K, H, T, r, sigma, q = 100.0, 100.0, 130.0, 1.0, 0.02, 0.20, 0.0
c_van = bs_price(S, K, T, r, sigma, q, kind='call')
c_uo  = call_up_and_out(S, K, H, T, r, sigma, q)
c_ui  = call_up_and_in (S, K, H, T, r, sigma, q)
float(c_van), float(c_uo), float(c_ui), abs(c_van - (c_uo + c_ui))  # deve essere ~ 0


(8.916037278572539, 3.128838772693303, 5.7871985058792355, 0.0)

In [21]:
# H < K -> up-and-out call deve valere 0 (se S < H)
S, K, H, T, r, sigma, q = 80.0, 100.0, 90.0, 1.0, 0.02, 0.20, 0.0
call_up_and_out(S, K, H, T, r, sigma, q)  # atteso: 0.0


0.0

In [28]:
# Parità (down): S>H e K>=H
importlib.reload(barrier_bs)
from barrier_bs import call_down_and_out, call_down_and_in

S, K, H, T, r, sigma, q = 100.0, 100.0, 80.0, 1.0, 0.02, 0.20, 0.0
c_van = bs_price(S, K, T, r, sigma, q, kind='call')
c_do  = call_down_and_out(S, K, H, T, r, sigma, q)
c_di  = call_down_and_in (S, K, H, T, r, sigma, q)
float(c_van), float(c_do), float(c_di), abs(c_van - (c_do + c_di))

# Limite H -> 0 (deve tornare ~ vanilla)
for H_try in [50.0, 20.0, 1e-6]:
    print(H_try, call_down_and_out(S, K, H_try, T, r, sigma, q))


50.0 6.910052347341967
20.0 6.935904609247999
1e-06 6.9359046092480625


In [29]:
S, K, H, T, r, sigma, q = 100.0, 100.0, 80.0, 1.0, 0.02, 0.20, 0.0
c_van = bs_price(S, K, T, r, sigma, q, kind='call')
c_do  = call_down_and_out(S, K, H, T, r, sigma, q)
c_di  = call_down_and_in (S, K, H, T, r, sigma, q)
float(c_van), float(c_do), float(c_di), c_van - (c_do + c_di)


(8.916037278572539, 1.8394265632638884, 7.07661071530865, 0.0)

In [30]:
importlib.reload(barrier_bs)
from barrier_bs import call_down_and_out, call_down_and_in
S, K, H, T, r, sigma, q = 100.0, 80.0, 90.0, 1.0, 0.02, 0.20, 0.0  # K < H < S
c_van = bs_price(S, K, T, r, sigma, q, kind='call')
c_do  = call_down_and_out(S, K, H, T, r, sigma, q)
c_di  = call_down_and_in (S, K, H, T, r, sigma, q)  # parity
print(c_van, c_do, c_di, c_van - (c_do + c_di))     # l’ultimo ~ 0


22.542853157065267 14.818648317710881 7.724204839354385 0.0


In [32]:
importlib.reload(barrier_bs)
from barrier_bs import put_down_and_out, put_down_and_in
# Parità down (put)
S, K, H, T, r, sigma, q = 100.0, 100.0, 80.0, 1.0, 0.02, 0.20, 0.0
p_van = bs_price(S, K, T, r, sigma, q, kind='put')
p_di  = put_down_and_in(S, K, H, T, r, sigma, q)
p_do  = put_down_and_out(S, K, H, T, r, sigma, q)
p_van, p_di, p_do, p_van - (p_di + p_do)   # l’ultimo ~ 0

# Sanity limit: H -> 0 (down-out put -> vanilla put)
for H_try in [50.0, 10.0, 1e-6]:
    print(H_try, put_down_and_out(S, K, H_try, T, r, sigma, q))


50.0 6.9345885236010245
10.0 6.93590460924807
1e-06 6.93590460924807


In [ ]:
# Setup autoreload consigliato
%load_ext autoreload
%autoreload 2

import sys, os, importlib, numpy as np
sys.path.insert(0, os.path.abspath(".."))

from src.bsm import bs_price
from src import barrier_bs
importlib.reload(barrier_bs)

from src.barrier_bs import (
    call_up_and_out, call_up_and_in,
    call_down_and_out, call_down_and_in,
    put_down_and_in, put_down_and_out,
    put_up_and_in, put_up_and_out
)

# --- Parità CALL (down)
S, K, H, T, r, sigma, q = 100.0, 100.0, 80.0, 1.0, 0.02, 0.20, 0.0
c_van = bs_price(S, K, T, r, sigma, q, kind='call')
c_do  = call_down_and_out(S, K, H, T, r, sigma, q)
c_di  = call_down_and_in (S, K, H, T, r, sigma, q)
print("CALL down parity diff:", c_van - (c_do + c_di))

# --- Parità PUT (down)
p_van = bs_price(S, K, T, r, sigma, q, kind='put')
p_di  = put_down_and_in(S, K, H, T, r, sigma, q)
p_do  = put_down_and_out(S, K, H, T, r, sigma, q)
print("PUT  down parity diff:", p_van - (p_di + p_do))

# --- Knock immediato
print("Knock immediate (up-out):", call_up_and_out(H, K, H, T, r, sigma, q))    # S=H -> 0
print("Knock immediate (down-out):", call_down_and_out(H, K, H, T, r, sigma, q))# S=H -> 0

# --- Limiti
for H_try in [50.0, 20.0, 1e-6]:
    print("H->0 DO call:", H_try, call_down_and_out(S, K, H_try, T, r, sigma, q))
    print("H->0 DO put :", H_try, put_down_and_out (S, K, H_try, T, r, sigma, q))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
CALL down parity diff: 0.0
PUT  down parity diff: 0.0
Knock immediate (up-out): 0.0
Knock immediate (down-out): 0.0
H->0 DO call: 50.0 8.916037278566714
H->0 DO put : 50.0 6.9345885236010245
H->0 DO call: 20.0 8.916037278572539
H->0 DO put : 20.0 6.93590460924807
H->0 DO call: 1e-06 8.916037278572539
H->0 DO put : 1e-06 6.93590460924807


In [35]:
# Setup autoreload consigliato
%load_ext autoreload
%autoreload 2

import sys, os, importlib, numpy as np
sys.path.insert(0, os.path.abspath(".."))

from src.bsm import bs_price
from src import barrier_bs
importlib.reload(barrier_bs)

from src.barrier_bs import (
    call_up_and_out, call_up_and_in,
    call_down_and_out, call_down_and_in,
    put_down_and_in, put_down_and_out,
    put_up_and_in, put_up_and_out
)
# Parità PUT (up)
S, K, H, T, r, sigma, q = 100.0, 100.0, 130.0, 1.0, 0.02, 0.20, 0.0
p_van = bs_price(S, K, T, r, sigma, q, kind='put')
p_ui  = put_up_and_in(S, K, H, T, r, sigma, q)
p_uo  = put_up_and_out(S, K, H, T, r, sigma, q)
print("PUT up parity diff:", p_van - (p_ui + p_uo))  # ~ 0

# Knock immediato: S>=H ⇒ UO=0, UI=vanilla
print("UO knock immediate:", put_up_and_out(H, K, H, T, r, sigma, q))   # 0.0
print("UI knock immediate:", put_up_and_in (H, K, H, T, r, sigma, q), " ~ ", bs_price(H, K, T, r, sigma, q, kind='put'))

# Limiti: H → ∞ ⇒ UO → vanilla put, UI → 0 (barriera irraggiungibile)
for H_try in [200.0, 500.0, 1e6]:
    print("H->∞ UO put:", H_try, put_up_and_out(S, K, H_try, T, r, sigma, q))
    print("H->∞ UI put:", H_try, put_up_and_in (S, K, H_try, T, r, sigma, q))


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
PUT up parity diff: 0.0
UO knock immediate: 0.0
UI knock immediate: 0.8036682103054638  ~  0.8036682103054638
H->∞ UO put: 200.0 308.91603727857796
H->∞ UI put: 200.0 -301.9801326693299
H->∞ UO put: 500.0 2408.916037278572
H->∞ UI put: 500.0 -2401.9801326693237
H->∞ UO put: 1000000.0 9999999908.916018
H->∞ UI put: 1000000.0 -9999999901.980112


In [37]:
import sys, os, importlib, numpy as np
sys.path.insert(0, os.path.abspath(".."))

from src.bsm import bs_price
from src import barrier_bs
importlib.reload(barrier_bs)

from src.barrier_bs import (
    call_up_and_out, call_up_and_in,
    call_down_and_out, call_down_and_in,
    put_down_and_in, put_down_and_out,
    put_up_and_in, put_up_and_out
)
S,K,H,T,r,sigma,q = 100,100,130,1.0,0.02,0.20,0.0
p_van = bs_price(S,K,T,r,sigma,q, kind="put")
p_uo  = put_up_and_out(S,K,H,T,r,sigma,q)
p_ui  = put_up_and_in (S,K,H,T,r,sigma,q)

print("PUT up parity:", p_van - (p_ui + p_uo))      # ~ 0
for H_try in [200, 500, 1e6]:
    print("H->∞ UO put:", H_try, put_up_and_out(S,K,H_try,T,r,sigma,q))  # → p_van ≈ 0.803668…
    print("H->∞ UI put:", H_try, put_up_and_in (S,K,H_try,T,r,sigma,q))  # → 0


PUT up parity: 0.0
H->∞ UO put: 200 6.935904609242549
H->∞ UI put: 200 5.5209170568559784e-12
H->∞ UO put: 500 6.93590460924807
H->∞ UI put: 500 0.0
H->∞ UO put: 1000000.0 6.93590460924807
H->∞ UI put: 1000000.0 0.0
